In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import opttrot


## Fast Pauli decomposition algorithm implementation

```
Hantzko, Binkowski, and Gupta, Tensorized Pauli decomposition algorithm
```

In [146]:
import numpy as np
from copy import deepcopy

In [147]:
from opttrot import utils 

In [148]:
# Size = 2^n Hermit matrix
n =5
size_n = int(2**n)
A = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
ImA = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
A = A.H@A
ImA = ImA.H@ImA
H = A+1j*ImA

In [149]:
pauli_matrix, pauli_terms = opttrot.Hamiltonian.generate_pauli_terms(n) # Get 2^n pauli-terms

In [150]:
# Frobenius inner product
decomposition = {}
for matrix, term in zip(pauli_matrix, pauli_terms):
    coef = utils.frobenius_inner(matrix, H)
    decomposition[term] = coef

In [151]:
decomposition

{'IIIII': (11.086474925671606+10.870156030559457j),
 'IIIIZ': (0.06535528417042968-0.14729443423261435j),
 'IIIZI': (-0.0760299172843526-0.06447764657056254j),
 'IIIZZ': (-0.3460163857217484+0.33631296420183654j),
 'IIZII': (0.5683124672027471+0.04916003183086409j),
 'IIZIZ': (0.26591170602794284+0.1605672889578143j),
 'IIZZI': (0.19096464693079523+0.15555698964134151j),
 'IIZZZ': (-0.39737536891735753-0.1511754317574277j),
 'IZIII': (0.2063759462745236-0.22652718539380923j),
 'IZIIZ': (0.18629292605781933-0.2739826731044799j),
 'IZIZI': (-0.016875523065415843-0.13748051737395417j),
 'IZIZZ': (0.08604014291145712-0.0881570561374202j),
 'IZZII': (0.3399881201690628-0.4030510816879932j),
 'IZZIZ': (-0.5281880355713504+0.25380699514628824j),
 'IZZZI': (-0.4299123112217155+0.33027868334626953j),
 'IZZZZ': (-0.3811614651493959-0.4077807047973634j),
 'ZIIII': (0.3360061812165268+0.042094228155063806j),
 'ZIIIZ': (-0.6929223063012235+0.08448450457121115j),
 'ZIIZI': (0.4346403398718479+0.2622

In [152]:
H_test = deepcopy(H)

### Implementation

In [153]:
exp2n = H_test.shape[0]
n = int(np.log2(exp2n))

print("Hamiltonian shape:", H_test.shape)
print("Total qubits:", n)


Hamiltonian shape: (32, 32)
Total qubits: 5


In [154]:
H_test = deepcopy(H)

In [155]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5


In [156]:
def index_xzcode(i, j):
    return i^j, i
def xzcode_index(nx, nz):
    return nz, nx^nz

In [157]:
opttrot.utils.pstr_from_xz(2, 4)

'ZXI'

In [158]:
nr, nc = H_test.shape
decom = {}
for i in range(nr):
    for j in range(nc):
        coef = H_test[i, j]
        decom[opttrot.utils.xz_fam_code_to_pstr(index_xzcode(i,j), n)] = coef
        

In [159]:
decom

{'IIIII': (11.086474925671606+10.870156030559457j),
 'IIIIX': (8.409658078200273+8.353720872238236j),
 'IIIXI': (8.207440475781741+8.269608290960267j),
 'IIIXX': (8.309610411277335+8.361306253383692j),
 'IIXII': (8.285748841344411+8.082728247237888j),
 'IIXIX': (8.361407540996941+8.314571008967695j),
 'IIXXI': (8.3329477264448+8.410045310801024j),
 'IIXXX': (8.289264143842887+8.564243082147106j),
 'IXIII': (8.393809720946024+8.325713942805201j),
 'IXIIX': (8.2512231408787+8.378195933473936j),
 'IXIXI': (8.635041335068305+8.433228430041485j),
 'IXIXX': (8.35064318797981+8.258091234859496j),
 'IXXII': (8.274746907883163+8.175191178823107j),
 'IXXIX': (8.347440146715432+8.27335181815635j),
 'IXXXI': (8.287473617919353+8.462168107754547j),
 'IXXXX': (8.182583472319555+8.306346658608035j),
 'XIIII': (8.525570657167634+8.29479011071863j),
 'XIIIX': (8.363759966610393+8.689137509448376j),
 'XIIXI': (8.345256914289767+8.164353601592774j),
 'XIIXX': (8.400641352146609+8.325037192262549j),
 'XIX

In [167]:
diff = {}
for key in decom.keys():
    diff[key] = abs(decomposition[key] - decom[key])

In [171]:
np.array(list(diff.values())).max()

3.972054645195637e-15

In [36]:
from pd import PauliDecomposition

In [37]:
pstring, pcoef = PauliDecomposition(H)
pdecompose = {st:coef for st, coef in zip(pstring, pcoef)}

In [38]:
pdecompose

{'II': (0.9005362940313093+1.3204313675811121j),
 'IX': (0.7635570833803782+0.9990637222435708j),
 'IZ': (-0.08622551788374494-0.08438509874824285j),
 'XI': (0.4870871213098663+0.9587796614603952j),
 'XX': (0.521002575605159+1.058572172370484j),
 'XZ': (-0.061228390080900724-0.00888796892719318j),
 'YY': (0.24851833341979765-0.17550013528666797j),
 'ZI': (-0.376079366498258-0.20722178207352765j),
 'ZX': (-0.3685600234469523-0.32519660297994957j),
 'ZZ': (-0.23338216083836252+0.3002352018351064j)}

In [14]:
def fast_p_decompose(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n))
    # Implementation of decomposition
    l = int(exp2n)
    for i in range(n):
        m = int(2**i)
        l = int(l/2)
        for j in range(m):
            for k in range(m):

                num_i = j*l
                num_j = k*l

                H[num_i: num_i+l, num_j:num_j+l] += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j

        H *= 0.5
    return H

# string to matrix

In [175]:
from opttrot.utils import pstr_to_matrix, pstr_to_xz_fam_code

In [176]:
pstr= "IZX"
xz_code = pstr_to_xz_fam_code(pstr)
p_matrix = pstr_to_matrix(pstr)

In [177]:
p_matrix

array([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -1.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,
        -0.+0.j]])

In [179]:
xz_code

(1, 2)

In [182]:
mat = np.zeros(p_matrix.shape)
mat[*xzcode_index(*xz_code)] = 1

In [ ]:
# Single Pauli term to matrix

In [ ]:
# Multiple Pauli term at once
# Inverse routine of decomposition

In [ ]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)
    #print("number of submatrix:", m)
    #print("sub matrix unit:", l)
    for j in range(m):
        for k in range(m):

            num_i = j*(2*l)
            num_j = k*(2*l)

            #print(f"{num_i}, {num_j}")
            #print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            #print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    #print("------------------------------------------------")
    H_test *= 0.5
